In [1]:
from heapq import *
from enum import Enum
import math
import copy
import numpy as np
import random

### Node


In [2]:
class NodeType(Enum):
    DEFAULT = 0
    OBSTACLE = 1
    PICKUP = 2
    DROPOFF = 3


class BasicNode(object):
    def __init__(self, new_id, new_type):
        self.id = new_id
        self.type = new_type

class AStarNode(BasicNode):
    def __init__(self, new_id, new_type, new_coordinaters):
        BasicNode.__init__(self, new_id, new_type)
        self.coordinates = new_coordinaters
        self.g = None
        self.h = None
        self.f = None
        self.came_from = None
        self.depth = 0
        self.booked = False

    def __lt__(self, other):
        if self.f == None or other.f == None:
            raise Exception("Some f value not initialized!")
        else:
            return self.f < other.f

    def __eq__(self, other):
        return (self.id == other.id and self.depth == other.depth)

    def __hash__(self):
        return self.id

    def compare(self, other):
        ok = True
        if self.id != other.id:
            print("1")
            return False
        if self.type != other.type:
            print("2")
            return False
        if self.coordinates != other.coordinates:
            print("3")
            return False
        if self.g != other.g:
            print("4")
            #return False
            ok = False
        if self.h != other.h:
            print("5")
            #return False
            ok = False
        if self.f != other.f:
            print("6")
            #return False
            ok = False
        if self.came_from and other.came_from:
            if self.came_from.id != other.came_from.id:
                print("7")
                ok = False
        elif self.came_from and not other.came_from:
            print("8")
            ok = False
        elif other.came_from and not self.came_from:
            print("9")
            ok = False
        if self.depth != other.depth:
            print("10")
            ok = False
        if self.booked != other.booked:
            print("11")
            ok = False

        if not ok:
            return False
        return True


### Agent

In [3]:
class Pickup(object):
    def __init__(self, item, worker):
        self.target_list = [item, worker, item]
        self.state = 0

    def other_worker_that_needs_item(self, workers, is_copy):
        for worker in workers:
            if worker.items and worker.items[0]:
                for item in worker.items[0]:
                    if item.id == self.target_list[0].id:
                        if not is_copy:
                            worker.remove_item(item)
                        return worker
        return None

    def advance_pickup_state(self, workers, is_copy):
        if self.state == 1:
            other_worker = self.other_worker_that_needs_item(workers, is_copy)
            if other_worker:
                self.target_list[1] = other_worker
                return True
            else:
                self.state += 1
                return True
        elif self.state == 2:
            self.target_list[0].booked = False
            return False
        else:
            self.state += 1
            return True



    def get_target(self):
        return self.target_list[self.state]

    def is_carrying_shelf(self):
        return self.state > 0

class Agent(object):
    def __init__(self, pos, id):
        self.id = id
        self.pos = pos
        self.path = []
        self.pickup = None
        self.walking_path = []
        self.target_path = []
        self.is_copy = False
        self.is_carrying_shelf = False
        self.was_at_target = False

    def copy(self):
        new_agent = Agent(self.pos, self.id)
        new_agent.path = self.path.copy()

        if self.pickup:
            # new_agent.pickup = Pickup(self.pickup.target_list[0], self.pickup.target_list[1])
            # new_agent.pickup.state = self.pickup.state
            new_agent.pickup = copy.deepcopy(self.pickup)

        new_agent.walking_path = self.walking_path.copy()
        new_agent.target_path = self.target_path.copy()
        new_agent.is_copy = self.is_copy
        new_agent.is_carrying_shelf = self.is_carrying_shelf
        new_agent.was_at_target = self.was_at_target

        return new_agent

    def move_on_path(self, steps, pickup):
        "This method is used in WHCA"

        if len(self.path) <= steps:
            self.pos = self.path[-1]
            self.walking_path += self.path[1:]
            if pickup:
                self.target_path += [pickup.target_list[0].id for x in self.path[1:]]
            else:
                self.target_path += [-1 for x in self.path[1:]]
        else:
            self.pos = self.path[steps]
            self.walking_path += self.path[1:steps+1]
            if pickup:
                self.target_path += [pickup.target_list[0].id for x in self.path[1:steps+1]]
            else:
                self.target_path += [-1 for x in self.path[1:steps+1]]
        # reset pos node for next interation
        self.pos.g = None
        self.pos.h = None
        self.pos.f = None
        self.pos.came_from = None
        self.pos.depth = 0

    def one_step_in_path(self, other=None):
        "This method is used in DKBR"

        if other:
            print("other in one step BEFORE POP is:")
            print([c.id for c in other.path])

        if len(self.path) > 1:
            self.path.pop(0)
            if other:
                print("other in one step is:")
                print([c.id for c in other.path])
            self.pos = self.path[0]
            #print("new pos is %d "% (self.pos.id))
            self.walking_path += [self.pos]
        elif len(self.path) > 0:
            self.pos = self.path[0]
            self.walking_path += [self.pos]
            #print("standing still at %d " % (self.pos.id))


    def done_with_target(self):
        if self.was_at_target:
            self.was_at_target = False
            return True
        elif self.pos.coordinates == self.pickup.get_target().coordinates:
            self.was_at_target = True
            return False
        else:
            self.was_at_target = False
            return False


### Functions

In [4]:
def get_item_size(order_input):
    size = 0
    for order in order_input:
        size += len(order)
    return size


def get_x_vector_from_state_first(state):
	x_vector = []
	x_vector.append(state.agent1.pos.id)
	x_vector.append(state.agent1.pickup.get_target().id)
	x_vector.append(state.agent2.pos.id)
	x_vector.append(state.agent2.pickup.get_target().id)
	for agent in state.agents:
		x_vector.append(agent.pos.id)
	return x_vector

def get_x_vector_from_state_area(state, graph):
	x_vector = []
	agent1 = state.agent1
	agent2 = state.agent2

	area_for_agent(x_vector, agent1, agent2, state.agents, graph)
	area_for_agent(x_vector, agent2, agent1, state.agents, graph)


	return x_vector

def area_for_agent(x_vector, agent1, agent2, agents, graph):
	neighbours = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
	for (i,j) in neighbours:

		x, y = (agent1.pos.coordinates[0] + i, agent1.pos.coordinates[1] + j)
		if x < 0 or x >= graph.shape[0] or y < 0 or y >= graph.shape[1]:
			# neighbour coordinates are out of the graph
			x_vector.append(1)
			continue

		neighbour = graph[x][y]

		if neighbour.coordinates == agent1.path[1].coordinates:
			x_vector.append(2)
			continue

		if neighbour.type == NodeType.OBSTACLE and agent1.is_carrying_shelf:
			x_vector.append(1)
			continue

		was_occupied = False
		for a in agents:
			if a.id == agent1.id or a.id == agent2.id:
				continue
			if len(a.path) > 1:
				if a.path[1].coordinates == neighbour.coordinates:
					x_vector.append(1)
					was_occupied = True
					break

		if was_occupied:
			continue

		x_vector.append(0)


def get_x_vector_from_state_coordinates(state):
	x_vector = []
	x_vector.append(state.agent1.pos.coordinates[0])
	x_vector.append(state.agent1.pos.coordinates[1])
	x_vector.append(state.agent1.pickup.get_target().coordinates[0])
	x_vector.append(state.agent1.pickup.get_target().coordinates[1])
	x_vector.append(state.agent2.pos.coordinates[0])
	x_vector.append(state.agent2.pos.coordinates[1])
	x_vector.append(state.agent2.pickup.get_target().coordinates[0])
	x_vector.append(state.agent2.pickup.get_target().coordinates[1])
	for agent in state.agents:
		x_vector.append(agent.pos.coordinates[0])
		x_vector.append(agent.pos.coordinates[1])
	return x_vector

def get_x_vector_from_state_coordinates_small(state):
	x_vector = []
	x_vector.append(state.agent1.pos.coordinates[0])
	x_vector.append(state.agent1.pos.coordinates[1])
	x_vector.append(state.agent1.path[1].coordinates[0])
	x_vector.append(state.agent1.path[1].coordinates[1])
	x_vector.append(state.agent2.pos.coordinates[0])
	x_vector.append(state.agent2.pos.coordinates[1])
	x_vector.append(state.agent2.path[1].coordinates[0])
	x_vector.append(state.agent2.path[1].coordinates[1])
	return x_vector


def write_line_to_file(x, file):
	for element in x:
		file.write(str(element) + " ")
	file.write("\n")

def read_lines_from_file(file):
    x = []
    lines = []
    for line in file:
        lines.append(line)

    for line in lines:
        elements = line.split()
        one_x = [int(elem) for elem in elements]
        x.append(one_x)
    return x

def get_correct_node(pickup_nodes, node_id):
    for current in pickup_nodes:
        if current.id == node_id:
            return current
    raise Exception('Could not find id:%d in the list of pickup_nodes' %(node_id))

def assign_first_items(agents, workers):
    for a in agents:
        assign_item_to_agent(a, workers)

def create_workers(drop_off_nodes):
    return [Worker(node.id, node.coordinates) for node in drop_off_nodes]

def create_orders(order_input, pickup_nodes):
    orders = []
    for order_list in order_input:
        order = []
        for node_id in order_list:
            order.append(get_correct_node(pickup_nodes, node_id))
        orders.append(order)
    return orders

def distribute_orders(workers, orders):
    worker_number = 0
    while orders:
        workers[worker_number].add_order(orders.pop(0))
        worker_number = (worker_number + 1) % len(workers)

def create_agents(drop_off_nodes, number_of_agents):
    agent_list = []
    for i in range(0, number_of_agents):
        agent_list.append(Agent(drop_off_nodes[i % len(drop_off_nodes)], i))
    return agent_list

def create_Astar_graph(warehouse):
    graph = np.ndarray((warehouse.shape), dtype=BasicNode)

    index = 0
    item_counter = 0
    workers = []
    items = []
    for (i,j), value in np.ndenumerate(warehouse):
            graph[i][j] = AStarNode(index, NodeType(value), (i,j))
            if value == 2:
                graph[i][j].booked = False
                items.append(graph[i][j])
            if value == 3:
                workers.append(graph[i][j])
            index += 1
    return graph, items, workers




# The heuristic used in A* to estimate the h-value
def manhattan_distance(start, end):
    dist_x = abs(start.coordinates[1] - end.coordinates[1])
    dist_y = abs(start.coordinates[0] - end.coordinates[0])
    return dist_x + dist_y


#Find the closest item.
def assign_item_to_agent(agent, workers):
    agent_pos = agent.pos
    min_dist = 10**5
    chosen_worker = None
    chosen_item = None
    for worker in workers:
        if worker.items and worker.items[0]:
            for item in worker.items[0]:
                if not item.booked:
                    dist = manhattan_distance(agent_pos, item)
                    if dist < min_dist:
                        min_dist = dist
                        chosen_worker = worker
                        chosen_item = item
    if chosen_worker:
        if not agent.is_copy:
            chosen_item.booked = True
        agent.pickup = Pickup(chosen_item, chosen_worker)
        if not agent.is_copy:
            chosen_worker.remove_item(chosen_item)
        return True
    agent.pickup = None
    return False

#하나라도 
def one_agent_has_pickup(agents):
    for a in agents:
        if a.pickup:
            return True
    return False

def reset_booked(graph, booked_items):
	# this might work since reset book should only happen
	# once a simulation is fully complete,
	# and the all nodes booked should be false.
	# for (x,y) in booked_items:
	# 	graph[x][y].booked = True
	for k in range(0, graph.shape[0]):
		for l in range(0, graph.shape[1]):
			if (k,l) in booked_items:
				graph[k][l].booked = True
			else:
				graph[k][l].booked = False

def reset_graph(graph):
    for i in range(0, graph.shape[0]):
        for j in range(0, graph.shape[1]):
            graph[i][j].g = None
            graph[i][j].h = None
            graph[i][j].f = None
            graph[i][j].came_from = None
            graph[i][j].depth = 0

def reset_f_val_graph(graph):
    for i in range(0, graph.shape[0]):
        for j in range(0, graph.shape[1]):
            graph[i][j].g = None
            graph[i][j].h = None
            graph[i][j].f = None
            graph[i][j].came_from = None

def extract_path(current, graph=None):
    path = [current]
    next_node = current.came_from
    while next_node:
        path.insert(0, next_node)
        next_node = next_node.came_from

    for node in path:
        node.came_from = None
    # if graph != None:
    #     reset_graph(graph)
    return path

### Worker

In [5]:
class Worker(object):
	def __init__(self, new_id, coordinates):
		self.items = []
		self.id = new_id
		self.coordinates = coordinates

	def add_order(self, order):
		self.items.append(order)

	def remove_item(self,item):
		self.items[0].remove(item)
		if not self.items[0]:
			self.items.pop(0)

### Astar

In [6]:
def AStar(graph, agent, p=False):
    #reset_graph(graph)
    start = agent.pos
    start.depth = 0
    start.came_from = None
    target = agent.pickup.get_target()

    if p:
        print("new astar from %d to %d" % (start.id, target.id))

    start.g = 0
    start.h = manhattan_distance(start, target)
    start.f = start.h

    neighbours = [(0,1),(1,0),(-1,0),(0,-1)]

    open_list = []
    closed_list = set()

    heappush(open_list, start) # add start to open list

    while open_list:
        current = heappop(open_list)
        closed_list.add(current)

        if current.coordinates == target.coordinates:
            # target is found, extract the path
            return extract_path(current, graph)

        for (i,j) in neighbours:
            x, y = (current.coordinates[0] + i, current.coordinates[1] + j)
            if x < 0 or x >= graph.shape[0] or y < 0 or y >= graph.shape[1]:
                # neighbour coordinates are out of the graph
                continue
            neighbour = graph[x][y]
            neighbour.depth = 0
            if neighbour.type == NodeType.OBSTACLE and agent.is_carrying_shelf:
                continue
            if neighbour in closed_list:
                continue

            if neighbour not in open_list or current.g + 1 < neighbour.g:
                neighbour.g = current.g + 1
                neighbour.h = manhattan_distance(neighbour, target)
                neighbour.f = neighbour.g + neighbour.h
                neighbour.came_from = current
                if neighbour not in open_list:
                    heappush(open_list, neighbour)

def collisionWillOccur(reservation_table, current, neighbour):
    if (neighbour.id, current.depth + 1) in reservation_table:
        # Moving to node collision
        return True
    if (current.id, current.depth + 1) in reservation_table and (neighbour.id, current.depth) in reservation_table:
        # Swap collision
        return True
    return False

def AStarReserved(graph, agent, reservation_table, p=False):
    #reset_graph(graph)
    start = agent.pos
    start.came_from = None
    start.depth = 0
    target = agent.pickup.get_target()

    if p:
        print("new astar from %d to %d" % (start.id, target.id))

    start.g = 0
    start.h = manhattan_distance(start, target)
    start.f = start.h

    neighbours = [(0,1),(1,0),(-1,0),(0,-1)]

    open_list = []
    closed_list = set()

    heappush(open_list, start) # add start to open list

    while open_list:
        current = heappop(open_list)
        closed_list.add((current.id, current.depth))

        if p:
            print("now looking at %d depth: %d" % (current.id, current.depth))

        if current.coordinates == target.coordinates:
            # target is found, extract the path
            path = extract_path(current, graph)
            if p:
                print("found path is")
                print([x.id for x in path])
            return path

        for (i,j) in neighbours:
            x, y = (current.coordinates[0] + i, current.coordinates[1] + j)
            if x < 0 or x >= graph.shape[0] or y < 0 or y >= graph.shape[1]:
                # neighbour coordinates are out of the graph
                continue
            neighbour = graph[x][y]
            #if not current.depth > 7:
            neighbour = copy.deepcopy(neighbour)
            neighbour.depth = current.depth + 1
            if neighbour.type == NodeType.OBSTACLE and agent.is_carrying_shelf:
                continue

            if (neighbour.id, current.depth + 1) in closed_list:
                if p:
                    print("skipped neighbour %d depth: %d becuase in CLOSED" % (neighbour.id, neighbour.depth))
                continue

            if collisionWillOccur(reservation_table, current, neighbour):
                if p:
                    print("skipped neighbour %d depth %d becuase of collision" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in open_list:
                # make neighbour refer to correct node object
                for x in open_list:
                    if x == neighbour:
                        neighbour = x
                        break

            if neighbour not in open_list or current.g + 1 < neighbour.g:
                neighbour.g = current.g + 1
                neighbour.h = manhattan_distance(neighbour, target)
                neighbour.f = neighbour.g + neighbour.h
                neighbour.came_from = current
                if neighbour not in open_list:
                    if p:
                        print("added %d depth %d to open list" % (neighbour.id, neighbour.depth))
                    heappush(open_list, neighbour)
    return []


In [7]:
warehouse = np.array(
[
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0],
[1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0],
[1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0],
[1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,1,2,1,2,1,2,1,2,1,2,1,2,1,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0],
[1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,2,1,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0],
[3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,1,2,1,2,1,2,1,2,1,2,1,2,1,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0],
[1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0],
[1,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,2,1,2,1,2,1,2,1,2,1,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,2,1,2,0,0,2,1,2,0,0,2,1,2,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,2,1,2,1,2,1,2,1,2,1,2,1,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
]
, dtype = np.int32)

In [8]:
left_pickups = [104,106,108,110,112,
            202,204,206,208,210,
            349,351,353,355,357,
            447,449,451,453,455,
            594,596,598,600,602,
            692,694,696,698,700,
            839,841,843,845,847,
            937,939,941,943,945,
            1084,1086,1088,1090,1092,
            1182,1184,1186,1188,1190]

#grahp, item, worker
_, temp, _ = create_Astar_graph(warehouse)
right_pickups = [x.id for x in temp if x.id not in left_pickups]

all_pickups = left_pickups + right_pickups


In [9]:
print(right_pickups)

[135, 137, 139, 141, 143, 166, 168, 171, 173, 176, 178, 233, 235, 237, 239, 241, 264, 266, 269, 271, 274, 276, 362, 364, 367, 369, 372, 374, 380, 382, 384, 386, 388, 460, 462, 465, 467, 470, 472, 478, 480, 482, 484, 486, 558, 560, 563, 565, 568, 570, 625, 627, 629, 631, 633, 723, 725, 727, 729, 731, 754, 756, 758, 760, 762, 764, 852, 854, 856, 858, 860, 862, 915, 917, 920, 922, 925, 927, 999, 1001, 1003, 1005, 1007, 1009, 1013, 1015, 1018, 1020, 1023, 1025, 1097, 1099, 1101, 1103, 1105, 1107, 1111, 1113, 1116, 1118, 1121, 1123, 1209, 1211, 1214, 1216, 1219, 1221, 1244, 1246, 1248, 1250, 1252, 1254, 1307, 1309, 1312, 1314, 1317, 1319, 1342, 1344, 1346, 1348, 1350, 1352]


In [10]:
def simulate_8020_orders(num_orders):
    orders = []
    for i in range(0, num_orders):
        current_order = []
        num_items = int(np.random.exponential()) + 1
        current_amount = 0
        while True:
            if random.random() > 0.2:
                current_order.append(left_pickups[random.randint(0, len(left_pickups)-1)])
            else:
                current_order.append(right_pickups[random.randint(0, len(right_pickups)-1)])

            current_amount += 1
            if current_amount == num_items:
                orders.append(current_order)
                break
    return orders

In [11]:
number_of_test = 10

In [12]:
def findPathAStar(graph, agent,start_pos, reservation_table, W, workers, K):
    # initialize starting node
    start = start_pos

    if not agent.pickup:
        return [start_pos]
    target = agent.pickup.get_target()

    start.g = 0
    start.h = manhattan_distance(start, target)
    start.f = start.h
    #start.depth = 0

    neighbours = [(0,1),(1,0),(-1,0),(0,-1),(0,0)]

    open_list = []
    closed_list = set()

    # add start to open list
    heappush(open_list, start)

    reached_target_last_step = False
    while open_list:
        current = heappop(open_list)
        closed_list.add(current)

        if current.depth == W:
            # target is found, extract the path
            return extract_path(current)

        #Check if reached goal
        if current.coordinates == target.coordinates:
            if not reached_target_last_step:
                reached_target_last_step = True
            else:
                path_so_far = extract_path(current)
                if len(path_so_far) > K + 1:
                    agent = copy.deepcopy(agent)
                    agent.is_copy = True

                if not agent.pickup.advance_pickup_state(workers, agent.is_copy):
                    #Delivered back shelf
                    if not assign_item_to_agent(agent, workers):
                        agent.pickup = None
                        agent.is_carrying_shelf = False
                if agent.pickup:
                    agent.is_carrying_shelf = agent.pickup.is_carrying_shelf()

                current_depth = current.depth
                reset_f_val_graph(graph)
                current.came_from = None
                next_path = findPathAStar(graph, agent, current, reservation_table, W, workers, K+1-len(path_so_far))
                if next_path == None:
                    return None
                return path_so_far + next_path[1:]
        else:
            reached_target_last_step = False

        for (i,j) in neighbours:
            x, y = (current.coordinates[0] + i, current.coordinates[1] + j)
            if x < 0 or x >= graph.shape[0] or y < 0 or y >= graph.shape[1]:
                # neighbour coordinates are out of the graph
                continue

            neighbour = graph[x][y]
            neighbour = copy.deepcopy(neighbour)
            neighbour.depth = current.depth + 1

            if neighbour.type == NodeType.OBSTACLE and agent.is_carrying_shelf:
                # neighbour is not traversable, an obstacle
                #print("neigbout %d depth %d has OBSTACLE" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in closed_list:
                # neighbour case has already been considered
                #print("neigbout %d depth %d has CLOSED" % (neighbour.id, neighbour.depth))
                continue

            if collisionWillOccur(reservation_table, current, neighbour):
                # can't move to neighbour since another agent is there
                #print("neigbout %d depth %d has colliiiishhh" % (neighbour.id, neighbour.depth))
                continue

            if neighbour in open_list:
                # make neighbour refer to correct node object
                for x in open_list:
                    if x == neighbour:
                        neighbour = x
                        break

            if neighbour not in open_list or current.g + 1 < neighbour.g:
                if neighbour.coordinates == target.coordinates:
                    # waiting at target is preferred
                    neighbour.g = current.g + 0
                else:
                    neighbour.g = current.g + 1
                neighbour.h = manhattan_distance(neighbour, target)
                neighbour.f = neighbour.g + neighbour.h

                # set parent
                neighbour.came_from = current

                if neighbour not in open_list:
                    # add newly discovered node to open list.
                    heappush(open_list, neighbour)

    print("IMPOSSIBLE PROBLEM")

In [13]:
def hash(a, b):
    return int((a+b)*(a+b+1)/2 + b)

def hash_inverse(value):
    a = value * 8
    b = a + 1
    c = math.sqrt(b)
    d = c - 1
    e = d/2
    w = int(e)

    f = w + 1
    g = w * f
    h = g/2
    y = value - h
    x = w - y
    return (x, y)

In [14]:
def WHCA(graph, agents, W, K, workers):

    reservation_table = dict()
    
    # 아직 해야할 작업이 존재
    while one_agent_has_pickup(agents):

        round_robin_shuffle(agents)

        # Find collision free paths for each agent
        for a in agents:
            if not a.pickup:
                a.path = []
                continue
            reset_graph(graph)
#             print(a.pos)
            path = findPathAStar(graph, a, a.pos, reservation_table, W, workers, K)

            if path == None:
                return -1

            # Reserve path
            for time, value in enumerate(path):
                if time <= W:
                    if value.type == NodeType.DROPOFF:
                        continue
                    reservation_table[hash(value.depth, value.id)] = True

            # Save path
            a.path = path

        for a in agents:
            if not a.pickup and not a.path:
                continue
            a.move_on_path(K, a.pickup)

        reservation_table = dict()

    total_cost = 0
    for agent in agents:
        total_cost += len(agent.walking_path)
    return total_cost

In [15]:
def round_robin_shuffle(agents):
    agents.append(agents.pop(0))

In [16]:
number_of_tests =10

In [17]:
def main():
    global number_of_tests
    cost_WHCA = []
    input_size_items = []
    input_size_orders = []
    number_of_agents = 20
    failed_count = 0
    impossible_problems_WHCA = 0
    for test_number in range(0,number_of_test):
        print("starting test {}".format(test_number))
        order_input = simulate_8020_orders(random.randint(2,15))
        current_item_size = get_item_size(order_input)
        current_order_size = len(order_input)
        graph_WHCA, pickup_nodes_WHCA, drop_off_nodes_WHCA = create_Astar_graph(warehouse)
        workers_WHCA = create_workers(drop_off_nodes_WHCA)
        orders_WHCA = create_orders(order_input, pickup_nodes_WHCA)
        distribute_orders(workers_WHCA, orders_WHCA)
        agents_WHCA = create_agents(drop_off_nodes_WHCA, number_of_agents)
        assign_first_items(agents_WHCA, workers_WHCA)
        one_algorithm_failed = False
        current_WHCA_COST = WHCA(graph_WHCA, agents_WHCA, 20, 10, workers_WHCA)
        if current_WHCA_COST == -1:
            impossible_problems_WHCA += 1
            one_algorithm_failed = True
        cost_WHCA.append(current_WHCA_COST)
        input_size_items.append(current_item_size)
        input_size_orders.append(current_order_size)
        print("order_input",order_input)
    number_of_tests = number_of_tests - failed_count
    if number_of_tests <= 0:
        print("No simulations were successful")
        return
    print("cost_WHCA",cost_WHCA)
    avg_cost_WHCA = np.sum(cost_WHCA) / number_of_tests
    print("The average cost for WHCA: %.3f" % (avg_cost_WHCA))
    print("impossible problems encountered: WHCA: %d" % (impossible_problems_WHCA))
        
        
            
        


In [18]:
if __name__ == "__main__":
    main()

starting test 0
order_input [[455], [692, 598], [847], [486], [1003], [925], [937, 602], [920, 945], [388, 1248], [698], [694], [1005], [839, 276, 945], [845, 698], [206]]
starting test 1
order_input [[449], [941], [600], [1092], [939, 210]]
starting test 2
order_input [[269, 455], [382, 1084], [210, 449], [208, 386], [210, 210, 106], [633, 600, 447, 351, 110, 939], [110], [1084, 1088, 602, 106], [700, 380, 106], [176, 700]]
starting test 3
order_input [[462, 349], [451, 208], [600], [1188, 202], [945], [462], [1182], [594], [208], [845], [937, 104], [847], [1188, 594, 729], [1317, 915], [349, 353, 941]]
starting test 4
order_input [[1090, 1123, 1309, 602], [1182], [856], [600], [1121], [692], [276], [453], [455], [351, 602, 941], [173, 451, 110], [839], [1092]]
starting test 5
order_input [[692, 467, 462], [176], [937], [106], [1086], [627, 1184, 447, 700]]
starting test 6
order_input [[754], [349], [104, 754], [941], [999, 451], [694, 945], [351], [1184, 204], [104]]
starting test 7
